In [2]:
### import libraries
import argparse
import json
import requests
from collections import defaultdict
import pandas as pd
import numpy as np

In [3]:
### credentials
# go to graphQL explorer & copy the token
# debug & extend the token - https://developers.facebook.com/tools/debug/accesstoken/?access_token=1188185338645694%7Clu6paxlAjEydJwExj0DCZqR-TRg&version=v16.0
ACCESS_TOKEN = "PUT KEY HERE"

In [4]:
### functions

def calculate_impressions_midpoint(data):
    """Calculate impressions midpoint"""
    # Convert from a string of characters into an integer
    upper_bound = int(data["impressions"]["upper_bound"])
    lower_bound = int(data["impressions"]["lower_bound"])

    return round((upper_bound + lower_bound) / 2)

def calculate_impressions_by_region(data, impressions):
    """Calculate impressions by regions"""
    # Create a new dict to contain the results
    result = {}

    # Loop through each region and calculate the impressions
    for chunk in data["delivery_by_region"]:
        result[chunk["region"]] = round(impressions * float(chunk["percentage"]))

    return result

def calculate_impressions_by_gender(data, impressions):
    """Calculates impressions by gender"""
    # Create a new dict to contain the results. The categories are pre-populated with 0s.
    results = {"male": 0, "female": 0, "unknown": 0}

    # Loop through demographic data
    for d in data["demographic_distribution"]:
        # Let's break this expression down:
        # 1. result[d["gender"]] looks for the key in the result dict that matches d["gender"]
        # 2. += is a short hand operator for addition and assignment. e.g. foo +=1 is the same
        #    as foo = foo + 1
        # 3. float(d["percentage"]) converts the JSON data into a decimal number (programmers
        #    call those floats because the decimal can move around).
        # 4. round(impressions * float(d["percentage"])) multiplies the percentage by the
        #    impressions and rounds up to the nearest integer
        results[d["gender"]] += round(impressions * float(d["percentage"]))

    return results

def calculate_impressions_by_age(data, impressions):
    """Calculate impressions by age"""
    # Create a new default dict to contain the results. The default dict makes any new element
    # default to a value specified by the user. In our case we're defaulting to an integer 0.
    # int is actually a function that retuns 0 when called with no arguments
    results = defaultdict(int)

    # Loop through the demographic data and add up the impressions
    for d in data["demographic_distribution"]:
        # Let's break this expression down:
        # 1. result[d["age"]] looks for the key in the result dict that matches d["age"]. If
        #    there is no key that matches d["age"] the default dict will make one with the value
        #    of int(), which is always 0.
        # 2. += is a short hand operator for addition and assignment. e.g. foo +=1 is the same
        #    as foo = foo + 1
        # 3. float(d["percentage"]) converts the JSON data into a decimal number (programmers
        #    call those floats because the decimal can move around).
        # 4. round(impressions * float(d["percentage"])) multiplies the percentage by the
        #    impressions and rounds up to the nearest integer
        results[d["age"]] += round(impressions * float(d["percentage"]))

    return results

In [5]:
### list of page IDs
idList = ['106039214814684',
'102281724942742',
'738063612887865',
'341751646428117',
'591566840920364',
'105502284969626',
'49560242814',
'101691091213750',
# '113891694102', removed Energy Citizens
'292970844058835',
'100801038449520',
'108095672108380',
'111394533709201',
'107500120800840',
'101242238726088',
'237209147160346',
'110124925319299',
'396341921119746',
'108203188195224',
'106656845034469',
'47710973068',
'482100658584410']

In [6]:
"""Entrypoint of the program"""

# Store the paginated data in here
data = []

# create dataframe
df = pd.DataFrame()

# for each item in the list, store their data in the pandas dataframe

# This might work for our request
for i in idList:
    print(i)
    response = requests.get("https://graph.facebook.com/v5.0/ads_archive", params={
    "access_token": ACCESS_TOKEN,
    "ad_type": "POLITICAL_AND_ISSUE_ADS",
    "ad_active_status": "ALL",
    "search_page_ids": i,
    "ad_reached_countries": ["US"],
    "ad_delivery_date_min": "2018-05-24",
    "ad_delivery_date_max": "2023-06-29",
    "fields": "id, ad_delivery_start_time, ad_delivery_stop_time, ad_snapshot_url, bylines, delivery_by_region, demographic_distribution, impressions, publisher_platforms, spend, ad_creative_bodies, ad_creative_link_captions, ad_creative_link_descriptions, ad_creative_link_titles, page_name, page_id"
})
    # Get the json document and pull out the next link and the data
    json = response.json()
    
    try:
        next_link = json['paging']['next']
        data = data + json['data']
    except KeyError:
        continue

    while next_link:
      print('.', end='')
      response = requests.get(next_link)
      json = response.json()
      if 'paging' not in json:
        break
      next_link = json['paging']['next']
      data = data + json['data']
    print('\n')

    # append data to dataframe
    res = pd.json_normalize(data)
    df.append(res)


106039214814684
.....

102281724942742
..........................................................................................

738063612887865
........................................................................................................................

341751646428117
..

591566840920364
............................................

105502284969626
.

49560242814
..........................................................................................

101691091213750
.........................................................................................

292970844058835
.

100801038449520
...............................................................................

108095672108380
..

111394533709201
.

107500120800840
.............

101242238726088
..............

237209147160346
..................

110124925319299
.

396341921119746
......

108203188195224
.........

106656845034469
........

47710973068
.........................................................

In [7]:
res

,id,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,bylines,delivery_by_region,demographic_distribution,publisher_platforms,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,page_name,page_id,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound
0,1343142396186676,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000545', 'region': 'Idaho'}...","[{'percentage': '0.000133', 'age': '13-17', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],[You Can Fight the Ban],Affordable Energy Coalition,106039214814684,20000,24999,600,699
1,619401349138540,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000434', 'region': 'Idaho'}...","[{'percentage': '0.000106', 'age': '13-17', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],[You Can Fight the Ban],Affordable Energy Coalition,106039214814684,25000,29999,800,899
2,879439772752678,2022-01-14,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.00037', 'region': 'Idaho'},...","[{'percentage': '0.000605', 'age': '18-24', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],[You Can Fight the Ban],Affordable Energy Coalition,106039214814684,15000,19999,0,99
3,1297817217381527,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '1', 'region': 'Washington'}]","[{'percentage': '0.000242', 'age': '18-24', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],[You Can Fight the Ban],Affordable Energy Coalition,106039214814684,7000,7999,0,99
4,3090071534604776,2022-01-14,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.00191', 'region': 'Idaho'},...","[{'percentage': '0.000526', 'age': '18-24', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],[You Can Fight the Ban],Affordable Energy Coalition,106039214814684,10000,14999,0,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17574,189303645276959,2018-07-31,2018-08-03,https://www.facebook.com/ads/archive/render_ad...,We Stand For Energy,"[{'percentage': '1', 'region': 'North Dakota'}]","[{'percentage': '0.001258', 'age': '18-24', 'g...","[facebook, instagram]",[North Dakota is leading the way in energy inn...,NaN,NaN,NaN,We Stand For Energy,482100658584410,0,999,0,99
17575,192309774968058,2018-07-31,2018-08-03,https://www.facebook.com/ads/archive/render_ad...,We Stand For Energy,"[{'percentage': '1', 'region': 'North Dakota'}]","[{'percentage': '0.000596', 'age': '18-24', 'g...","[facebook, instagram]",[Help keep energy bills low by supporting a br...,NaN,NaN,NaN,We Stand For Energy,482100658584410,1000,1999,0,99
17576,263101847818994,2018-07-31,2018-08-03,https://www.facebook.com/ads/archive/render_ad...,We Stand For Energy,"[{'percentage': '1', 'region': 'North Dakota'}]","[{'percentage': '0.001269', 'age': '18-24', 'g...","[facebook, instagram]",[Wind energy helps create jobs in North Dakota...,NaN,NaN,NaN,We Stand For Energy,482100658584410,4000,4999,0,99
17577,1031653150337021,2018-05-09,2018-06-10,https://www.facebook.com/ads/archive/render_ad...,NaN,"[{'percentage': '1', 'region': 'Virginia'}]","[{'percentage': '0.001071', 'age': '18-24', 'g...","[facebook, instagram, audience_network, messen...",[The Army Corps of Engineers has signed off on...,[WeS

### Clean & Export Data

### For Summary Table

In [8]:
# check NaNs:
# res['ad_delivery_stop_time'][res['ad_delivery_stop_time'].isna()]

# fill NaN with '999'
res['ad_delivery_stop_time'] = res['ad_delivery_stop_time'].fillna('999')

In [9]:
# create 'ad start month' and 'ad start year' column
res['ad_start_month'] = res['ad_delivery_start_time'].str.split('-').str[1].astype(int)
res['ad_start_year'] = res['ad_delivery_start_time'].str.split('-').str[0].astype(int)

In [10]:
# convert string to integer
res['spend.lower_bound'] = res['spend.lower_bound'].astype('int')
res['spend.upper_bound'] = res['spend.upper_bound'].astype('int').round(-2)

In [11]:
# title case for bylines column
res['bylines'] = res['bylines'].str.title()

In [12]:
res[0:1]

,id,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,bylines,delivery_by_region,demographic_distribution,publisher_platforms,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,page_name,page_id,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound,ad_start_month,ad_start_year
0,1343142396186676,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000545', 'region': 'Idaho'}...","[{'percentage': '0.000133', 'age': '13-17', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],[You Can Fight the Ban],Affordable Energy Coalition,106039214814684,20000,24999,600,700,1,2022


In [13]:
# export raw data
res.to_csv("../data/all-front-group-ads-062923.csv")

In [14]:
resjson = res[['page_name', 'id', 'spend.lower_bound', 'spend.upper_bound']].groupby('page_name').agg({'id':'count', 'spend.lower_bound': 'sum', 'spend.upper_bound': 'sum'}).reset_index().rename(columns={'page_name':'name', 'id':'ads', 'spend.lower_bound': 'lowerAmount', 'spend.upper_bound':'upperAmount'}).sort_values(by="upperAmount", ascending=False).reset_index(drop=True)

In [15]:
resjson

,name,ads,lowerAmount,upperAmount
0,Voices for Cooperative Power,1671,998700,1367800
1,Natural Allies for a Clean Energy Future,1965,852200,1211100
2,The Empowerment Alliance,2205,590000,924000
3,Consumer Energy Alliance,2229,583100,922500
4,Alliance for Michigan Power,2977,219300,541200
5,Citizens Energizing Michigan's Economy,1100,237300,378700
6,Seniors Across America,212,248300,324500
7,Affordable Energy for New Jersey,2233,94700,323600
8,Power for Tomorrow,326,220000,290500
9,Partnership for Energy Progress,317,178400,242600


In [16]:
# export data
resjson.to_json('../data/summary-062923.json', orient = 'records')

### For Individual Tables

In [17]:
# unpack the dataset to access region data
result = pd.concat([res.explode('delivery_by_region').drop(['delivery_by_region'], axis=1),
           res.explode('delivery_by_region')['delivery_by_region'].apply(pd.Series)],
          axis=1)

In [18]:
# convert to float type 
result['percentage'] = result['percentage'].astype(float)

In [20]:
# calculate lower & upper amount spent per region
result['lowerAmount'] = result['percentage'] * result['spend.lower_bound']
result['upperAmount'] = result['percentage'] * result['spend.upper_bound']

In [21]:
# view data
result[['page_name', 'region', 'lowerAmount', 'upperAmount']].groupby(['page_name', 'region']).sum(['lowerAmount', 'upperAmount']).reset_index().rename(columns={'page_name':'name'})

,name,region,lowerAmount,upperAmount
0,Affordable Energy Coalition,British Columbia,0.0940,0.1410
1,Affordable Energy Coalition,California,0.1195,0.1337
2,Affordable Energy Coalition,Colorado,0.1080,0.1260
3,Affordable Energy Coalition,Idaho,94.9952,136.0257
4,Affordable Energy Coalition,Indiana,0.0700,0.1050
...,...,...,...,...
2043,We Stand For Energy,Washington,454.1054,1056.5721
2044,We Stand For Energy,"Washington, District of Columbia",766.3988,1629.3098
2045,We Stand For Energy,West Virginia,439.9411,877.5189
2046,We Stand For Energy,Wisconsin,380.4889,826.4896


In [22]:
# if the percentage amount is less 10%, don't include it
result.query("percentage >.10")
# result.query(" page_name == 'Cooperative Action Network' ")

,id,ad_delivery_start_time,ad_delivery_stop_time,ad_snapshot_url,bylines,demographic_distribution,publisher_platforms,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,...,impressions.upper_bound,spend.lower_bound,spend.upper_bound,ad_start_month,ad_start_year,0,percentage,region,lowerAmount,upperAmount
0,1343142396186676,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000133', 'age': '13-17', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],...,24999,600,700,1,2022,NaN,0.999455,Washington,599.6730,699.6185
1,619401349138540,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000106', 'age': '13-17', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],...,29999,800,900,1,2022,NaN,0.999566,Washington,799.6528,899.6094
2,879439772752678,2022-01-14,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000605', 'age': '18-24', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],...,19999,0,100,1,2022,NaN,0.999630,Washington,0.0000,99.9630
3,1297817217381527,2022-01-13,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000242', 'age': '18-24', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],...,7999,0,100,1,2022,NaN,1.000000,Washington,0.0000,100.0000
4,3090071534604776,2022-01-14,2022-05-31,https://www.facebook.com/ads/archive/render_ad...,Affordable Energy Coalition,"[{'percentage': '0.000526', 'age': '18-24', 'g...","[facebook, instagram]",[Middle-class families are being pushed out of...,[affordableenergycoalition.org],[Tell the City What’s at Stake],...,14999,0,100,1,2022,NaN,0.998090,Washington,0.0000,99.8090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17574,189303645276959,2018-07-31,2018-08-03,https://www.facebook.com/ads/archive/render_ad...,We Stand For Energy,"[{'percentage': '0.001258', 'age': '18-24', 'g...","[facebook, instagram]",[North Dakota is leading the way in energy inn...,NaN,NaN,...,999,0,100,7,2018,NaN,1.000000,North Dakota,0.0000,100.0000
17575,192309774968058,2018-07-31,2018-08-03,https://www.facebook.com/ads/archive/render_ad...,We Stand For Energy,"[{'percentage': '0.000596', 'age': '18-24', 'g...","[facebook, instagram]",[Help keep energy bills low by supporting a br...,NaN,NaN,...,1999,0,100,7,2018,NaN,1.000000,North Dakota,0.0000,100.0000
17576,263101847818994,2018-07-31,2018-08-03,https://www.facebook.com/ads/archive/render_ad...,We Stand For Energy,"[{'percentage': '0.001269', 'age': '18-24', 'g...","[facebook, instagram]",[Wind energy helps create jobs in North Dakota...,NaN,NaN,...,4999,0,100,7,2018,NaN,1.000000,North Dakota,0.0000,100.0000
17577,1031653150337021,2018-05-09,2018-06-10,https://www.facebook.com/ads/archive/render_ad...,NaN,"[{'percentage': '0.001071', 'age': '18-24', 'g...","[facebook, instagram, audience_network, messen...",[The Army Corps of Engineers has signed off on...,[WeStandForEnergy.com],[Tell the State Water Board: We Want Jobs!],...,59999,1000,1500,5,2018,NaN,1.000000,Virginia,1000.0000,1500.0000


In [23]:
# export data
result[['page_name', 'region', 'lowerAmount', 'upperAmount', 'percentage']].query("percentage > 0.10").groupby(['page_name', 'region']).sum(['lowerAmount', 'upperAmount']).reset_index().rename(columns={'page_name':'name'}).to_json('../data/individual-062923.json', orient = 'records')